In [1]:
%matplotlib widget
from helper import *

# Application Example: Robot Welding

A good example where such motion controllers are needed is robotic welding. This section will briefly show the combination of the newly derived motion controller and the newly derived interpolation algorithm and an example  via the developed simulation environment. To incorperate the extra redundancy given by the symmetry around the welding tool, the cobot kinematics are extended by an extra degree of freedom. This joint is then used virtually as an extra DoF for optimization. Modern welding applications with cobots are often intuitively set up with a hand-guiding mode. Here, the welding gun is positioned in the key poses on the welding part by the user, and motion primitives are chosen for interpolation between these key poses. 

A more modern approach developed by Neura Robotics is that the welding part and the welding seams are predefined by the user, uploaded in the GUI as virtual model. With the virtual model and the advanced object detection of Maira, the position and orientation of the welding part, which can be arbitrarily placed in some predefined area, can be detected. The parts are then welded autonomously. The attached video will showcase this.

In [2]:
%%HTML
<video width="1100" height="700" controls>
  <source src=".\Resources\maira_welding.mp4" type="video/mp4">
</video>

In both cases, key poses as a set of unit dual quaternions $\mathbb{X} = [\underline{\xi}_1, \underline{\xi}_2, \dots, \underline{\xi}_n]$ are intrinsically defined, either via the forward kinematics of the robot when using the hand guiding mode, or via the object detection algorithm and the predefined key poses in the given CAD file. In terms of this showcase, the key poses are defined manually. The application of dual quaternions in robot welding is especially interesting as it allows to discern between short and long-path interpolation of orientation. This is beneficial as non-expert users often struggle with conceptualizing a short path orientation interpolation, as would be needed for a homogenous transformation matrix-based approach. Here, no double cover of $\mathcal{SE}(3)$ is given, which forces the roboticists to fall back on strict short-path interpolation, even if the intention by the user was different during the setup of the robotic trajectory.

The following, manually defined key poses $\mathbb{X}$ define different sets of poses which are either used for linear, or circular interpolation by leveraging the interpolation algorithm of <a href="./3.4_dual_quaternion_interpolation.ipynb">Chapter 3.4</a>.

In [3]:
# orientations for first trajectory
quat0 = Quaternion.fromAxisAngle(np.pi*1.25, np.array([0,1,0]))
quat1 = Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,0,1]))
quat2 = Quaternion.fromAxisAngle(np.pi, np.array([0,0,1]))
quat3 = Quaternion.fromAxisAngle(np.pi*1.5, np.array([0,0,1]))

# first line segments
x1 = DualQuaternion.fromQuatPos(quat0, np.array([1.2, -0.3, 0.1]))
x2 = DualQuaternion.fromQuatPos(quat0, np.array([1.2, 0.1, 0.1]))
x3 = DualQuaternion.fromQuatPos(quat1*quat0, np.array([1.2, 0.1, 0.1]))
x4 = DualQuaternion.fromQuatPos(quat1*quat0, np.array([0.950, 0.1, 0.1]))
x5 = DualQuaternion.fromQuatPos(quat2*quat0, np.array([0.950, 0.1, 0.1]))
x6 = DualQuaternion.fromQuatPos(quat2*quat0, np.array([0.950, -0.3, 0.1]))
x7 = DualQuaternion.fromQuatPos(quat3*quat0, np.array([0.950, -0.3, 0.1]))
x8 = DualQuaternion.fromQuatPos(quat3*quat0, np.array([1.2, -0.3, 0.1]))

# orientations for second trajectory
quat4 = Quaternion.fromAxisAngle(np.pi*1.5, np.array([0,1,0]))
quat5 = Quaternion.fromAxisAngle(np.pi*0.25, np.array([0,0,1])) 
quat6 = Quaternion.fromAxisAngle(-np.pi*0.5, np.array([0,0,1])) 
quat7 = Quaternion.fromAxisAngle(-np.pi, np.array([0,0,1])) 

# second line segments
x9 = DualQuaternion.fromQuatPos(quat4, np.array([0.850, 0.0, 0.1]))
x10 = DualQuaternion.fromQuatPos(quat4, np.array([0.850, 0.0, 0.4]))
x11 = DualQuaternion.fromQuatPos(quat5*quat0, np.array([0.850, 0.0, 0.4]))

#first arc segments
x12 = DualQuaternion.fromQuatPos(quat0, np.array([0.90, -0.05, 0.4]))
x13 = DualQuaternion.fromQuatPos(quat6*quat0, np.array([0.850, -0.1, 0.4]))
x14 = DualQuaternion.fromQuatPos(quat7*quat0, np.array([0.80, -0.05, 0.4]))

# third line segments
x15 = DualQuaternion.fromQuatPos(quat7*quat0, np.array([0.80, 0.3, 0.4]))
x16 = DualQuaternion.fromQuatPos(quat6*quat0, np.array([0.80, 0.3, 0.4]))
x17 = DualQuaternion.fromQuatPos(quat6*quat0, np.array([0.60, 0.3, 0.4]))
x18 = DualQuaternion.fromQuatPos(quat6*quat0, np.array([0.60, 0.0, 0.6]))

For an intuitive setup, the required unit dual quaternions are defined as orientation unit quaternion and desired position, respectively.

Given the desired motion limits and the desired cartesian velocity for the welding task as defined in the following code block:

In [4]:
# define the acceleration and jerk limits
a_cart_max = 3
j_cart_max = 30
a_ang_max = 3
j_ang_max = 30

# define desired cartesian velocites and maximum angular velocity
des_cart_vel = 0.03
max_ang_vel = 1.5

The desired trajectories from the DQQB interpolation algorithm are then defined as known from <a href="./3.4_dual_quaternion_interpolation.ipynb">Chapter 3.4</a>. For this, we define the required sets of unit dual quaternions $\mathbb{X}_{line1}$, $\mathbb{X}_{line2}$, $\mathbb{X}_{arc2}$ and $\mathbb{X}_{line3}$. These sets are used to define two trajectories. The first trajectory moves along the unit dual quaternion set $\mathbb{X}_{line1}$, defined by the LineGenerator. The second trajectory concatenates the last three unit dual quaternion sets to produce a mix of MoveLinear and MoveCircle, which results in a MoveComposite motion.

In [5]:
# assign dual quaternions to lists
line1_DQ_list = [x1, x2, x3, x4, x5, x6]
line2_DQ_list = [x9, x10, x11]
arc2_DQ_list = [x11, x12, x13, x14]
line3_DQ_list = [x14, x15, x16, x17]

# initialize the DQQBTrajectoryGenerator
trajectory1 = DQQBTrajectoryGenerator()
trajectory2 = DQQBTrajectoryGenerator()

# initialize line and arc generators
line_generator = LineGenerator()
arc_generator = ArcGenerator()

# generate segments
line_segments1 = line_generator.generateSegments(line1_DQ_list, des_cart_vel, max_ang_vel)
line_segments2 = line_generator.generateSegments(line2_DQ_list, des_cart_vel, max_ang_vel)
arc_segments2 = arc_generator.generateSegments(arc2_DQ_list, des_cart_vel, max_ang_vel)
line_segments3 = line_generator.generateSegments(line3_DQ_list, des_cart_vel, max_ang_vel)

segments2 = line_segments2 + arc_segments2 + line_segments3

trajectory1.generateDynamicTrajectory(line_segments1, a_cart_max, j_cart_max, a_ang_max, j_ang_max)
trajectory2.generateDynamicTrajectory(segments2, a_cart_max, j_cart_max, a_ang_max, j_ang_max)

iteration:  1
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  1
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
blendphases overlapped
iteration:  2
blendphases overlapped
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10


Both trajectories successfully ran the offline optimization, as seen acceleration limits were violated and blend phases overlapped. The entire robot motion is set up as a concatenation of <i>tasks</i>. These tasks also include MoveJoint and MoveLinear commands, which connect the continous paths from the DQQB interpolation. The robot type can be defined as either "weld" or "extended" defining MAiRA as a 7 or 8 DoF robot, depending on whether the virtual joint should be activated to enable another DoF or not.

In [6]:
robot_type = "weld"
#robot_type = "extended"

fk = ForwardKinematics(robot_type)

q1 = np.array([deg2rad(-20), deg2rad(40), 0, deg2rad(70), 0, deg2rad(40), 0])
q2 = np.array([deg2rad(10), deg2rad(30), 0, deg2rad(90), 0, deg2rad(40), 0])

task_list = np.array([MoveJoint(q1, 2), MoveLinear(fk.getFK(q1), x1, 1), 
                      MoveTrajectory(trajectory1), MoveLinear(x6, fk.getFK(q1), 3),
                      MoveJoint(q2, 4), MoveLinear(fk.getFK(q2), x9, 2),
                      MoveTrajectory(trajectory2), MoveLinear(x17, x18, 1)])

Running the last code block sets up the simulation environment and starts the simulated welding process. The motion controller chosen can be "classic" or "qp", where either the classic formulation via pseudo inversion of the Jacobian with nullspace projection of the manipulability gradient is shown, or the developed predictive motion controller from <a href="./3.3_predictive_differential_kinematics.ipynb">Chapter 3.3</a>. The demo shown here shows the proposed QP-based motion controller. The reader is encouraged to test the "classic" motion controller and the "extended" robot type, as well as their perturbations with the current settings. 

In [8]:
control_method = "qp"
#control_method = "classic"

sim = Simulation(task_list, robot_type, method = control_method)

sim.start()

As seen in the Simulation environment, the proposed method and interpolation scheme is capable of complex path interpolation and the ability to smoothly follow this trajectory in an online fashion.